In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Função para implementar o Bagging do zero
class BaggingClassifier:
    def __init__(self, base_model, n_estimators=10):
        self.base_model = base_model
        self.n_estimators = n_estimators
        self.models = []
    
    def fit(self, X, y):
        # Treinando múltiplos modelos em diferentes subconjuntos de dados
        n_samples = X.shape[0]
        for _ in range(self.n_estimators):
            # Amostragem com reposição (Bootstrap sampling)
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_bootstrap = X[indices]
            y_bootstrap = y[indices]
            
            # Treinando uma árvore de decisão em cada subconjunto
            model = self.base_model
            model.fit(X_bootstrap, y_bootstrap)
            self.models.append(model)
    
    def predict(self, X):
        # Fazendo previsões usando votação por maioria (para classificação)
        predictions = np.zeros((self.n_estimators, X.shape[0]), dtype=int)  # Garantir que o tipo é inteiro
        for i, model in enumerate(self.models):
            predictions[i] = model.predict(X).astype(int)  # Garantir que as previsões são inteiras
        
        # Votação por maioria
        return np.array([np.bincount(pred).argmax() for pred in predictions.T])

# Carregar conjunto de dados Iris
iris = load_iris()
X = iris.data
y = iris.target

# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------
# Treinando uma única árvore de decisão
# -----------------------
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Previsões e avaliação da árvore única
y_pred_tree = tree_model.predict(X_test)
tree_accuracy = accuracy_score(y_test, y_pred_tree)
print(f"Acurácia da única árvore de decisão: {tree_accuracy:.4f}")

# -----------------------
# Treinando o modelo Bagging
# -----------------------
bagging_model = BaggingClassifier(base_model=DecisionTreeClassifier(random_state=42), n_estimators=50)
bagging_model.fit(X_train, y_train)

# Previsões e avaliação do modelo Bagging
y_pred_bagging = bagging_model.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)
print(f"Acurácia do modelo Bagging: {bagging_accuracy:.4f}")


Acurácia da única árvore de decisão: 1.0000
Acurácia do modelo Bagging: 1.0000
